#Distributed ML Algorithms #1 - MNIST
###Here is a basic ML algorithm trained on the MNIST dataset with a CNN-model, trained in distributed fashion on multiple GPU(s).

###The model utilizes available functions from tensorflow.


In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

##Number of Available GPU(s)

In [2]:
print("Available number of GPU(s):", len(tf.config.list_physical_devices('GPU')))

Available number of GPU(s): 1


##Loading Data from MNIST

In [3]:
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

###Normalizing data

X_train, X_test = X_train/255.0, X_test/255.0

11490434/11490434 [==============================] - 1s 0us/step


##Defining a CNN-Model-Creating Function

In [4]:
def create_model():
  model = models.Sequential()
  model.add(layers.Reshape(input_shape = (28,28), target_shape = (28,28,1)))
  model.add(layers.Conv2D(32, (3,3), activation = 'relu'))
  model.add(layers.MaxPooling2D(2,2))
  model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
  model.add(layers.MaxPooling2D(2,2))
  model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation = 'relu'))
  model.add(layers.Dense(10))

  return model

##Distributing the Training with Tensorflow:

In [5]:
#Mirrored Strategy Creation:

strategy = tf.distribute.MirroredStrategy()
print("Number of Devices:", strategy.num_replicas_in_sync)

#Now perform under the strategy scope:

with strategy.scope():
  #Everything that creates variables is defined under this section.
  #Therefore, the model 'installation', 'compilation' and all others occur here.
  model = create_model()
  model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), optimizer = "adam", metrics = ['accuracy'])

Number of Devices: 1


##Fit the model and get the results

In [6]:
###As the model is created within the strategy's scope, all the GPU(s) start fitting the model when the 'fit' method is called as below:
model.fit(X_train, y_train, epochs = 5)


###Now evaluate the model:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose = 2)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Epoch 1/5
1875/1875 [==============================] - 19s 4ms/step - loss: 0.1398 - accuracy: 0.9561
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0443 - accuracy: 0.9865
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0325 - accuracy: 0.9893
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0260 - accuracy: 0.9914
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0210 - accuracy: 0.9931
313/313 - 1s - loss: 0.0438 - accuracy: 0.9854 - 1s/epoch - 4ms/step
Test Loss: 0.04379551485180855
Test Accuracy: 0.9854000210762024
